In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, optimizers
import tensorflow as tf
from joblib import Parallel, delayed
from collections import defaultdict
import hashlib
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [ ]:
data1 = pd.read_excel("КеЕ.xlsx")
df_MOEX = pd.DataFrame(data1)
df_MOEX = df_MOEX.sort_values(by="Дата")
df_MOEX["Дата"] = pd.to_datetime(df_MOEX["Дата"])

data2 = pd.read_csv("spy.csv")
df_SPY = pd.DataFrame(data2)
df_SPY["Date"] = pd.to_datetime(df_SPY["Date"])

data3 = pd.read_excel("dollar.xlsx")
df_dollar = pd.DataFrame(data3)
df_dollar = df_dollar.sort_values(by="Дата")
df_dollar["Дата"] = pd.to_datetime(df_dollar["Дата"])

data4 = pd.read_excel("hang_seng.xlsx")
df_HKE = pd.DataFrame(data4)
df_HKE["Дата"] = pd.to_datetime(df_HKE["Дата"])


data5 = pd.read_excel("gold.xlsx")
df_gold = pd.DataFrame(data5)
df_gold["Дата"] = pd.to_datetime(df_gold["Дата"])


data6 = pd.read_excel("usd_hkd.xlsx")
df_usd_hkd = pd.DataFrame(data6)
df_usd_hkd["Дата"] = pd.to_datetime(df_usd_hkd["Дата"])

start_date = "1998-09-23"
end_date = "2017-11-10"

df_MOEX = df_MOEX[(df_MOEX["Дата"] >= start_date) & (df_MOEX["Дата"] <= end_date)]
df_SPY = df_SPY[(df_SPY["Date"] >= start_date) & (df_SPY["Date"] <= end_date)]
df_dollar = df_dollar[(df_dollar["Дата"] >= start_date) & (df_dollar["Дата"] <= end_date)]
df_HKE = df_HKE[(df_HKE["Дата"] >= start_date) & (df_HKE["Дата"] <= end_date)]
df_GOLD = df_gold[(df_gold["Дата"] >= start_date) & (df_gold["Дата"] <= end_date)]

df_SPY.head()

,Date,Open,High,Low,Close,Volume,Day,Weekday,Week,Month,Year
1427,1998-09-23,65.282020,67.245979,65.223101,67.245979,13688500,23,2,39,9,1998
1428,1998-09-24,66.813861,67.128094,64.889182,65.596207,11129500,24,3,39,9,1998
1429,1998-09-25,64.810660,66.254170,64.496426,65.517685,10247400,25,4,39,9,1998
1430,1998-09-28,66.185423,66.813890,65.517677,66.106865,8847800,28,0,40,9,1998
1431,1998-09-29,66.224684,66.578197,64.967751,65.949730,10618100,29,1,40,9,1998


In [ ]:
# Рассчитаем цену MOEX, HKE в долларах и доходности в процентах
df_MOEX["Цена доллары"] = df_MOEX["Цена"] / df_dollar["Цена"]
df_MOEX['Returns_in_dollars'] = df_MOEX['Цена доллары'].pct_change() * 100
df_MOEX = df_MOEX.dropna(subset=['Returns_in_dollars'])

df_HKE["Цена доллары"] = df_HKE["Цена"] / df_usd_hkd["Цена"]
df_HKE['Returns_in_dollars'] = df_HKE['Цена доллары'].pct_change() * 100
df_HKE = df_HKE.dropna(subset=['Returns_in_dollars'])

df_SPY['Returns'] = df_SPY['Close'].pct_change() * 100
df_SPY = df_SPY.dropna(subset=['Returns'])

df_GOLD['Returns'] = df_GOLD['Цена'].pct_change() * 100
df_GOLD = df_GOLD.dropna(subset=['Returns'])

In [ ]:
# Функция для расчёта исторического VaR по торговой доходности
def calculate_historical_VaR(returns_series, portfolio_value=1000000, confidence_level=0.95):
    sorted_returns = returns_series.sort_values().reset_index(drop=True)
    N = len(sorted_returns)
    # Позиция (5-й процентиль при 95%-м уровне)
    position = int((1 - confidence_level) * N - 1)
    position = max(0, position)
    var_quantile = sorted_returns.iloc[position]
    VaR = -var_quantile * portfolio_value / 100
    return VaR

# Расчёт исторического VaR для MOEX
VaR_MOEX = calculate_historical_VaR(df_MOEX['Returns_in_dollars'])
print(f"Historical VaR (MOEX) на уровне {0.95*100}%: {VaR_MOEX:.2f} USD")

# Расчёт исторического VaR для HKE
VaR_HKE = calculate_historical_VaR(df_HKE['Returns_in_dollars'])
print(f"Historical VaR (HKE) на уровне {0.95*100}%: {VaR_HKE:.2f} USD")

# Расчёт исторического VaR для SPY
VaR_SPY = calculate_historical_VaR(df_SPY['Returns'])
print(f"Historical VaR (SPY) на уровне {0.95*100}%: {VaR_SPY:.2f} USD")

# Расчёт исторического VaR для фьючерсов на золото
VaR_gold = calculate_historical_VaR(df_GOLD['Returns'])
print(f"Historical VaR (Gold) на уровне {0.95*100}%: {VaR_gold:.2f} USD")

Historical VaR (MOEX) на уровне 95.0%: 35256.70 USD
Historical VaR (HKE) на уровне 95.0%: 22820.71 USD
Historical VaR (SPY) на уровне 95.0%: 19343.03 USD
Historical VaR (Gold) на уровне 95.0%: 17305.98 USD


In [ ]:
def calculate_parametric_VaR(df, price_col, date_col, portfolio_value=1000000, confidence_level=0.95, tolerance=0.01):
    # Рассчитываем скользящие средние (MA200 и EMA200)
    df['MA200'] = df[price_col].rolling(window=200).mean()
    df['EMA200'] = df[price_col].ewm(span=200, adjust=False).mean()

    # Сигнал покупки: цена близка к MA200 (с учетом tolerance) и предыдущая цена выше MA200
    df['signal_buy'] = ((abs(df[price_col] - df['MA200']) / df['MA200'] < tolerance) &
                        (df[price_col].shift(1) > df['MA200']))

    trade_returns = []
    for idx, row in df[df['signal_buy']].iterrows():
        purchase_date = row[date_col]
        purchase_price = row[price_col]
        end_date = purchase_date + pd.Timedelta(days=365)
        mask = (df[date_col] >= purchase_date) & (df[date_col] <= end_date)
        period_data = df.loc[mask]
        if period_data.empty:
            continue
        max_price = period_data[price_col].max()
        trade_return = (max_price - purchase_price) / purchase_price
        trade_returns.append(trade_return)

    trade_returns = np.array(trade_returns)
    mu = trade_returns.mean() if trade_returns.size > 0 else 0
    sigma = trade_returns.std() if trade_returns.size > 0 else 0
    z = norm.ppf(confidence_level)
    critical_return = mu - z * sigma
    VaR = -critical_return * portfolio_value
    df_name = df.name if hasattr(df, 'name') else "DataFrame"
    print("Parametric VaR (95%) for Index", df_name, VaR)

# Присвоим имена для DataFrame
df_MOEX.name = "MOEX"
df_SPY.name = "SPY"
df_HKE.name = "HKE"
df_GOLD.name = "GOLD"

# Расчет параметрического VaR для индексов и золота
calculate_parametric_VaR(df_MOEX, "Цена доллары", "Дата")
calculate_parametric_VaR(df_SPY, "Close", "Date")
calculate_parametric_VaR(df_HKE, "Цена доллары", "Дата")
calculate_parametric_VaR(df_GOLD, "Цена", "Дата")

Parametric VaR (95%) for Index MOEX 453233.6227494818
Parametric VaR (95%) for Index SPY 14664.865723031939
Parametric VaR (95%) for Index HKE -37085.66443110628
Parametric VaR (95%) for Index GOLD -39598.44314124965


In [ ]:
def calculate_trade_stats(df, price_col, date_col, tolerance=0.01):
    # Рассчет 200-дневного простого скользящего среднего
    df['MA200'] = df[price_col].rolling(window=200).mean()

    # Сигнал покупки: цена близка к MA200 и предыдущая цена выше MA200
    df['signal_buy'] = ((abs(df[price_col] - df['MA200']) / df['MA200'] < tolerance) &
                        (df[price_col].shift(1) > df['MA200']))

    trade_returns = []
    for idx, row in df[df['signal_buy']].iterrows():
        purchase_date = row[date_col]
        purchase_price = row[price_col]
        end_date = purchase_date + pd.Timedelta(days=365)
        mask = (df[date_col] >= purchase_date) & (df[date_col] <= end_date)
        period_data = df.loc[mask]
        if period_data.empty:
            continue
        max_price = period_data[price_col].max()
        trade_return = (max_price - purchase_price) / purchase_price
        trade_returns.append(trade_return)

    trade_returns = np.array(trade_returns)
    mu = trade_returns.mean() if trade_returns.size > 0 else 0
    sigma = trade_returns.std() if trade_returns.size > 0 else 0
    return mu, sigma

# Вычисление статистик для MOEX и SPY
mu_MOEX, sigma_MOEX = calculate_trade_stats(df_MOEX.copy(), "Цена доллары", "Дата", tolerance=0.01)
mu_SPY, sigma_SPY = calculate_trade_stats(df_SPY.copy(), "Close", "Date", tolerance=0.01)
mu_HKE, sigma_HKE = calculate_trade_stats(df_HKE.copy(), "Цена доллары", "Дата", tolerance=0.01)
mu_GOLD, sigma_GOLD = calculate_trade_stats(df_GOLD.copy(), "Цена", "Дата", tolerance=0.01)

print("MOEX: μ =", mu_MOEX, "σ =", sigma_MOEX)
print("SPY:  μ =", mu_SPY, "σ =", sigma_SPY)
print("HKE: μ =", mu_HKE, "σ =", sigma_HKE)
print("GOLD:  μ =", mu_GOLD, "σ =", sigma_GOLD)

MOEX: μ = 0.3150633795605348 σ = 0.4670914114917069
SPY:  μ = 0.14946398789164142 σ = 0.09978325786888734
HKE: μ = 0.12580373778216705 σ = 0.05393675880782686
GOLD:  μ = 0.11088006771217856 σ = 0.043336150647666064


In [ ]:
# Ожидаемый доход в денежном выражении
portfolio_value = 10**6
expected_profit_MOEX = mu_MOEX * portfolio_value
expected_profit_SPY  = mu_SPY * portfolio_value
expected_profit_HKE = mu_HKE * portfolio_value
expected_profit_GOLD  = mu_GOLD * portfolio_value

print("Ожидаемый доход при вложении 1 000 000$:\n")
print("MOEX:", expected_profit_MOEX)
print("SPY: ", expected_profit_SPY)
print("HKE:", expected_profit_HKE)
print("GOLD: ", expected_profit_GOLD)

Ожидаемый доход при вложении 1 000 000$:

MOEX: 315063.37956053484
SPY:  149463.98789164142
HKE: 125803.73778216705
GOLD:  110880.06771217857


In [ ]:
# Перебор вариантов распределения портфеля
confidence_level = 0.95
z = norm.ppf(confidence_level)  # z ≈ 1.645
results = []
list1 = []

# Перебор всех возможных вариантов распределения весов
for w_MOEX in range(1, 97):  # Оставляем место для HKE и GOLD
    for w_SPY in range(1, 98 - w_MOEX):
        for w_HKE in range(1, 99 - w_MOEX - w_SPY):
            w_GOLD = 100 - w_MOEX - w_SPY - w_HKE  # Остаток уходит на GOLD

            # Преобразуем в десятичные доли
            w_MOEX_dec = w_MOEX / 100.0
            w_SPY_dec = w_SPY / 100.0
            w_HKE_dec = w_HKE / 100.0
            w_GOLD_dec = w_GOLD / 100.0

            # Средняя доходность портфеля
            mu_portfolio = (w_MOEX_dec * mu_MOEX +
                            w_SPY_dec * mu_SPY +
                            w_HKE_dec * mu_HKE +
                            w_GOLD_dec * mu_GOLD)

            # Стандартное отклонение портфеля (допустим, активы не коррелированы)
            sigma_portfolio = np.sqrt(
                (w_MOEX_dec * sigma_MOEX) ** 2 +
                (w_SPY_dec * sigma_SPY) ** 2 +
                (w_HKE_dec * sigma_HKE) ** 2 +
                (w_GOLD_dec * sigma_GOLD) ** 2
            )

            # Расчет VaR
            critical_return = mu_portfolio - z * sigma_portfolio
            VaR_portfolio = -critical_return * portfolio_value
            expected_profit_portfolio = mu_portfolio * portfolio_value
            k = expected_profit_portfolio - VaR_portfolio

            # Сохраняем результаты
            list1.append((k, w_MOEX, w_SPY, w_HKE, w_GOLD))
            results.append((w_MOEX, w_SPY, w_HKE, w_GOLD, VaR_portfolio, expected_profit_portfolio, k))

# # Вывод результатов
# print("\nРезультаты для распределения портфеля (параметрический подход):")
# for res in results:
#     print(f"MOEX: {res[0]}%, SPY: {res[1]}%, HKE: {res[2]}%, GOLD: {res[3]}%, "
#           f"VaR: {res[4]:.2f}, ожидаемый доход: {res[5]:.2f}, доход-VaR: {res[6]:.2f}")

# Определение лучшего варианта
max_k, best_w_MOEX, best_w_SPY, best_w_HKE, best_w_GOLD = max(list1, key=lambda x: x[0])
print("\nМаксимально минимально ожидаемый доход:", f"{max_k:.2f}",
      "\nпри соотношении MOEX:", f"{best_w_MOEX}%", "SPY:", f"{best_w_SPY}%",
      "HKE:", f"{best_w_HKE}%", "GOLD:", f"{best_w_GOLD}%")


Максимально минимально ожидаемый доход: 203704.53 
при соотношении MOEX: 5% SPY: 25% HKE: 44% GOLD: 26%


In [ ]:
# Число итераций для Монте-Карло
iterations = 100
alpha_percentile = (1 - confidence_level) * 100

# Прогноз VaR для каждого актива отдельно
sim_returns_MOEX = np.random.normal(mu_MOEX, sigma_MOEX, iterations)
VaR_MOEX_MC = -np.percentile(sim_returns_MOEX, alpha_percentile) * portfolio_value

sim_returns_SPY = np.random.normal(mu_SPY, sigma_SPY, iterations)
VaR_SPY_MC = -np.percentile(sim_returns_SPY, alpha_percentile) * portfolio_value

sim_returns_HKE = np.random.normal(mu_HKE, sigma_HKE, iterations)
VaR_HKE_MC = -np.percentile(sim_returns_HKE, alpha_percentile) * portfolio_value

sim_returns_GOLD = np.random.normal(mu_GOLD, sigma_GOLD, iterations)
VaR_GOLD_MC = -np.percentile(sim_returns_GOLD, alpha_percentile) * portfolio_value

print("\nМонте-Карло прогнозирование VaR для отдельных активов:")
print(f"MOEX: ожидаемый доход = {mu_MOEX * portfolio_value:.2f}, VaR = {VaR_MOEX_MC:.2f}")
print(f"SPY:  ожидаемый доход = {mu_SPY * portfolio_value:.2f}, VaR = {VaR_SPY_MC:.2f}")
print(f"HKE:  ожидаемый доход = {mu_HKE * portfolio_value:.2f}, VaR = {VaR_HKE_MC:.2f}")
print(f"GOLD: ожидаемый доход = {mu_GOLD * portfolio_value:.2f}, VaR = {VaR_GOLD_MC:.2f}")

# Прогноз VaR для портфеля по Монте-Карло
results_mc = []
mc_results = []
for w_MOEX in range(1, 97):  # Оставляем место для HKE и GOLD
    for w_SPY in range(1, 98 - w_MOEX):
        for w_HKE in range(1, 99 - w_MOEX - w_SPY):
            w_GOLD = 100 - w_MOEX - w_SPY - w_HKE  # Остаток уходит на GOLD

            # Преобразуем в десятичные доли
            w_MOEX_dec = w_MOEX / 100.0
            w_SPY_dec = w_SPY / 100.0
            w_HKE_dec = w_HKE / 100.0
            w_GOLD_dec = w_GOLD / 100.0

            # Генерация случайных доходностей
            sim_MOEX = np.random.normal(mu_MOEX, sigma_MOEX, iterations)
            sim_SPY = np.random.normal(mu_SPY, sigma_SPY, iterations)
            sim_HKE = np.random.normal(mu_HKE, sigma_HKE, iterations)
            sim_GOLD = np.random.normal(mu_GOLD, sigma_GOLD, iterations)

            # Моделирование доходности портфеля
            sim_portfolio = (w_MOEX_dec * sim_MOEX +
                             w_SPY_dec * sim_SPY +
                             w_HKE_dec * sim_HKE +
                             w_GOLD_dec * sim_GOLD)

            VaR_portfolio_MC = -np.percentile(sim_portfolio, alpha_percentile) * portfolio_value
            mu_portfolio = (w_MOEX_dec * mu_MOEX +
                            w_SPY_dec * mu_SPY +
                            w_HKE_dec * mu_HKE +
                            w_GOLD_dec * mu_GOLD)
            expected_profit_portfolio = mu_portfolio * portfolio_value
            k = expected_profit_portfolio - VaR_portfolio_MC

            results_mc.append((w_MOEX, w_SPY, w_HKE, w_GOLD, VaR_portfolio_MC, expected_profit_portfolio, k))
            mc_results.append((k, w_MOEX, w_SPY, w_HKE, w_GOLD))

# print("\nМонте-Карло прогнозирование VaR для портфеля:")
# for res in results_mc:
#     print(f"MOEX: {res[0]}%, SPY: {res[1]}%, HKE: {res[2]}%, GOLD: {res[3]}%, "
#           f"VaR: {res[4]:.2f}, ожидаемый доход: {res[5]:.2f}, доход-VaR: {res[6]:.2f}")

# Оптимальное распределение
max_k_mc, best_w_MOEX_mc, best_w_SPY_mc, best_w_HKE_mc, best_w_GOLD_mc = max(mc_results, key=lambda x: x[0])
print("\nОптимальное распределение по методу Монте-Карло:")
print(f"Максимально минимально ожидаемый доход: {max_k_mc:.2f} достигается при соотношении "
      f"MOEX: {best_w_MOEX_mc}%, SPY: {best_w_SPY_mc}%, HKE: {best_w_HKE_mc}%, GOLD: {best_w_GOLD_mc}%")



Монте-Карло прогнозирование VaR для отдельных активов:
MOEX: ожидаемый доход = 315063.38, VaR = 622346.39
SPY:  ожидаемый доход = 149463.99, VaR = 19608.82
HKE:  ожидаемый доход = 125803.74, VaR = -43199.52
GOLD: ожидаемый доход = 110880.07, VaR = -35973.39

Оптимальное распределение по методу Монте-Карло:
Максимально минимально ожидаемый доход: 248655.52 достигается при соотношении MOEX: 12%, SPY: 36%, HKE: 24%, GOLD: 28%


In [ ]:
def get_trade_returns(df, price_col, date_col, tolerance=0.01):
    """
    Вычисляет торговые доходности по стратегии:
    - 200-дневное скользящее среднее (MA200)
    - Сигнал покупки: цена близка к MA200 (± tolerance) и предыдущая цена выше MA200
    - Для каждого сигнала покупки ищется максимальная цена в течение 365 дней
    """
    df = df.copy()
    df['MA200'] = df[price_col].rolling(window=200).mean()
    df['signal_buy'] = ((abs(df[price_col] - df['MA200']) / df['MA200'] < tolerance) &
                        (df[price_col].shift(1) > df['MA200']))
    trade_returns = []
    for idx, row in df[df['signal_buy']].iterrows():
        purchase_date = row[date_col]
        purchase_price = row[price_col]
        end_date = purchase_date + pd.Timedelta(days=365)
        mask = (df[date_col] >= purchase_date) & (df[date_col] <= end_date)
        period_data = df.loc[mask]
        if period_data.empty:
            continue
        max_price = period_data[price_col].max()
        ret = (max_price - purchase_price) / purchase_price
        trade_returns.append(ret)
    return np.array(trade_returns)

def historical_CVaR(trade_returns, portfolio_value, confidence_level=0.95):
    """
    Рассчитывает исторический CVaR:
    - Находим VaR как процентиль (например, 5-й процентиль при 95%-м уровне)
    - Затем CVaR – отрицательное среднее доходностей, ниже VaR, умноженное на стоимость портфеля
    """
    if trade_returns.size == 0:
        return 0
    alpha_percentile = (1 - confidence_level) * 100
    VaR_level = np.percentile(trade_returns, alpha_percentile)
    tail_losses = trade_returns[trade_returns <= VaR_level]
    if tail_losses.size == 0:
        return 0
    CVaR = -np.mean(tail_losses) * portfolio_value
    return CVaR

In [ ]:
# Получаем торговые доходности для всех активов
trade_returns_MOEX = get_trade_returns(df_MOEX.copy(), "Цена доллары", "Дата", tolerance=0.01)
trade_returns_SPY  = get_trade_returns(df_SPY.copy(), "Close", "Date", tolerance=0.01)
trade_returns_HKE  = get_trade_returns(df_HKE.copy(), "Цена доллары", "Дата", tolerance=0.01)
trade_returns_GOLD = get_trade_returns(df_GOLD.copy(), "Цена", "Дата", tolerance=0.01)

# Рассчитываем CVaR для каждого актива
CVaR_MOEX = historical_CVaR(trade_returns_MOEX, portfolio_value)
CVaR_SPY  = historical_CVaR(trade_returns_SPY, portfolio_value)
CVaR_HKE  = historical_CVaR(trade_returns_HKE, portfolio_value)
CVaR_GOLD = historical_CVaR(trade_returns_GOLD, portfolio_value)

# Средняя доходность по торговым данным
mu_trade_MOEX = trade_returns_MOEX.mean() if trade_returns_MOEX.size > 0 else 0
mu_trade_SPY  = trade_returns_SPY.mean() if trade_returns_SPY.size > 0 else 0
mu_trade_HKE  = trade_returns_HKE.mean() if trade_returns_HKE.size > 0 else 0
mu_trade_GOLD = trade_returns_GOLD.mean() if trade_returns_GOLD.size > 0 else 0

# Ожидаемый доход
expected_profit_MOEX = mu_trade_MOEX * portfolio_value
expected_profit_SPY  = mu_trade_SPY * portfolio_value
expected_profit_HKE  = mu_trade_HKE * portfolio_value
expected_profit_GOLD = mu_trade_GOLD * portfolio_value

# Вывод результатов
print("Исторический метод (по торговым данным):")
print(f"MOEX: μ = {mu_trade_MOEX:.6f} -> ожидаемый доход = {expected_profit_MOEX:.2f}, CVaR = {CVaR_MOEX:.2f}")
print(f"SPY:  μ = {mu_trade_SPY:.6f} -> ожидаемый доход = {expected_profit_SPY:.2f}, CVaR = {CVaR_SPY:.2f}")
print(f"HKE:  μ = {mu_trade_HKE:.6f} -> ожидаемый доход = {expected_profit_HKE:.2f}, CVaR = {CVaR_HKE:.2f}")
print(f"GOLD: μ = {mu_trade_GOLD:.6f} -> ожидаемый доход = {expected_profit_GOLD:.2f}, CVaR = {CVaR_GOLD:.2f}")


# Получаем торговые доходности для MOEX и SPY
trade_returns_MOEX = get_trade_returns(df_MOEX.copy(), "Цена доллары", "Дата", tolerance=0.01)
trade_returns_SPY  = get_trade_returns(df_SPY.copy(), "Close", "Date", tolerance=0.01)

CVaR_MOEX = historical_CVaR(trade_returns_MOEX, portfolio_value)
CVaR_SPY  = historical_CVaR(trade_returns_SPY, portfolio_value)

mu_trade_MOEX = trade_returns_MOEX.mean() if trade_returns_MOEX.size > 0 else 0
mu_trade_SPY  = trade_returns_SPY.mean() if trade_returns_SPY.size > 0 else 0

expected_profit_MOEX = mu_trade_MOEX * portfolio_value
expected_profit_SPY  = mu_trade_SPY * portfolio_value

Исторический метод (по торговым данным):
MOEX: μ = 0.315063 -> ожидаемый доход = 315063.38, CVaR = -0.00
SPY:  μ = 0.149464 -> ожидаемый доход = 149463.99, CVaR = -0.00
HKE:  μ = 0.125804 -> ожидаемый доход = 125803.74, CVaR = -60715.69
GOLD: μ = 0.110880 -> ожидаемый доход = 110880.07, CVaR = -62898.26


In [ ]:
# Бутстрепинг для портфеля
iterations = 100
bootstrap_results = []
results_bootstrap = []

# Если массивы пусты, зададим нулевые значения
if trade_returns_MOEX.size == 0:
    trade_returns_MOEX = np.array([0])
if trade_returns_SPY.size == 0:
    trade_returns_SPY = np.array([0])
if trade_returns_HKE.size == 0:
    trade_returns_HKE = np.array([0])
if trade_returns_GOLD.size == 0:
    trade_returns_GOLD = np.array([0])

# Перебираем возможные веса активов в портфеле
for w_MOEX in range(1, 97):
    for w_SPY in range(1, 98 - w_MOEX):
        for w_HKE in range(1, 99 - w_MOEX - w_SPY):
            w_GOLD = 100 - w_MOEX - w_SPY - w_HKE

            w_MOEX_dec = w_MOEX / 100.0
            w_SPY_dec  = w_SPY / 100.0
            w_HKE_dec  = w_HKE / 100.0
            w_GOLD_dec = w_GOLD / 100.0

            portfolio_returns = []

            for _ in range(iterations):
                sample_MOEX = np.random.choice(trade_returns_MOEX)
                sample_SPY  = np.random.choice(trade_returns_SPY)
                sample_HKE  = np.random.choice(trade_returns_HKE)
                sample_GOLD = np.random.choice(trade_returns_GOLD)

                port_ret = (
                    w_MOEX_dec * sample_MOEX +
                    w_SPY_dec * sample_SPY +
                    w_HKE_dec * sample_HKE +
                    w_GOLD_dec * sample_GOLD
                )
                portfolio_returns.append(port_ret)

            portfolio_returns = np.array(portfolio_returns)

            VaR_port = np.percentile(portfolio_returns, alpha_percentile)
            tail_losses = portfolio_returns[portfolio_returns <= VaR_port]
            CVaR_port = -np.mean(tail_losses) * portfolio_value if tail_losses.size > 0 else 0

            mu_portfolio = (
                w_MOEX_dec * mu_trade_MOEX +
                w_SPY_dec * mu_trade_SPY +
                w_HKE_dec * mu_trade_HKE +
                w_GOLD_dec * mu_trade_GOLD
            )

            expected_profit_portfolio = mu_portfolio * portfolio_value
            k = expected_profit_portfolio - CVaR_port

            results_bootstrap.append((w_MOEX, w_SPY, w_HKE, w_GOLD, CVaR_port, expected_profit_portfolio, k))
            bootstrap_results.append((k, w_MOEX, w_SPY, w_HKE, w_GOLD))

# # Вывод результатов
# print("\nИсторическое прогнозирование CVaR (бутстрепинг) для портфеля:")
# for res in results_bootstrap:
#     print(f"MOEX: {res[0]}%, SPY: {res[1]}%, HKE: {res[2]}%, GOLD: {res[3]}%, "
#           f"CVaR: {res[4]:.2f}, ожидаемый доход: {res[5]:.2f}, доход-CVaR: {res[6]:.2f}")

# Поиск оптимального распределения
max_k_bs, best_w_MOEX_bs, best_w_SPY_bs, best_w_HKE_bs, best_w_GOLD_bs = max(bootstrap_results, key=lambda x: x[0])

print("\nОптимальное распределение по историческому методу (CVaR):")
print(f"Максимально минимально ожидаемый доход: {max_k_bs:.2f} достигается при соотношении:")
print(f"MOEX: {best_w_MOEX_bs}%, SPY: {best_w_SPY_bs}%, HKE: {best_w_HKE_bs}%, GOLD: {best_w_GOLD_bs}%")


Оптимальное распределение по историческому методу (CVaR):
Максимально минимально ожидаемый доход: 313873.55 достигается при соотношении:
MOEX: 94%, SPY: 3%, HKE: 1%, GOLD: 2%


In [ ]:
z = norm.ppf(confidence_level)
phi_z = norm.pdf(z)

results_param_CVaR = []

# Перебираем возможные распределения портфеля
for w_MOEX in range(1, 97):
    for w_SPY in range(1, 98 - w_MOEX):
        for w_HKE in range(1, 99 - w_MOEX - w_SPY):
            w_GOLD = 100 - w_MOEX - w_SPY - w_HKE

            w_MOEX_dec = w_MOEX / 100.0
            w_SPY_dec  = w_SPY / 100.0
            w_HKE_dec  = w_HKE / 100.0
            w_GOLD_dec = w_GOLD / 100.0

            mu_portfolio = (
                w_MOEX_dec * mu_MOEX +
                w_SPY_dec * mu_SPY +
                w_HKE_dec * mu_HKE +
                w_GOLD_dec * mu_GOLD
            )

            sigma_portfolio = np.sqrt(
                (w_MOEX_dec * sigma_MOEX) ** 2 +
                (w_SPY_dec * sigma_SPY) ** 2 +
                (w_HKE_dec * sigma_HKE) ** 2 +
                (w_GOLD_dec * sigma_GOLD) ** 2
            )

            # Расчёт параметрического CVaR для портфеля
            CVaR_portfolio = - (mu_portfolio - sigma_portfolio * (phi_z / (1 - confidence_level))) * portfolio_value
            results_param_CVaR.append((w_MOEX, w_SPY, w_HKE, w_GOLD, CVaR_portfolio))

# # Вывод результатов
# print("Параметрический CVaR для разных распределений портфеля:")
# for res in results_param_CVaR:
#     print(f"MOEX: {res[0]}%, SPY: {res[1]}%, HKE: {res[2]}%, GOLD: {res[3]}%, CVaR: {res[4]:.2f}")

# Поиск оптимального распределения
optimal = min(results_param_CVaR, key=lambda x: x[4])

print("\nОптимальное распределение по параметрическому CVaR:")
print(f"MOEX: {optimal[0]}%, SPY: {optimal[1]}%, HKE: {optimal[2]}%, GOLD: {optimal[3]}% с CVaR: {optimal[4]:.2f}")



Оптимальное распределение по параметрическому CVaR:
MOEX: 2%, SPY: 15%, HKE: 38%, GOLD: 45% с CVaR: -57647.63


In [ ]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
from arch import arch_model
import numpy as np
import pandas as pd

def fit_garch_model(returns):
    # Обучает GARCH(1,1) модель на исторических доходностях и возвращает условные волатильности.
    model = arch_model(returns, vol='Garch', p=1, q=1, rescale=False)
    model_fit = model.fit(update_freq=0, disp='off')
    return model_fit.conditional_volatility

# Добавляем условную волатильность в данные MOEX, SPY, HKE и GOLD
df_MOEX['conditional_volatility'] = fit_garch_model(df_MOEX['Returns_in_dollars'].dropna())
df_SPY['conditional_volatility'] = fit_garch_model(df_SPY['Returns'].dropna())
df_HKE['conditional_volatility'] = fit_garch_model(df_HKE['Returns_in_dollars'].dropna())
df_GOLD['conditional_volatility'] = fit_garch_model(df_GOLD['Returns'].dropna())

# Функция для получения доходности и волатильности по сделкам
def get_trade_returns_with_vol(df, price_col, date_col, vol_col, tolerance=0.01):
    df = df.copy()
    df['MA200'] = df[price_col].rolling(window=200).mean()
    df['signal_buy'] = ((abs(df[price_col] - df['MA200']) / df['MA200'] < tolerance) &
                        (df[price_col].shift(1) > df['MA200']))
    trade_data = []
    for idx, row in df[df['signal_buy']].iterrows():
        purchase_date = row[date_col]
        purchase_price = row[price_col]
        end_date = purchase_date + pd.Timedelta(days=365)
        mask = (df[date_col] >= purchase_date) & (df[date_col] <= end_date)
        period_data = df.loc[mask]
        if period_data.empty:
            continue
        max_price = period_data[price_col].max()
        ret = (max_price - purchase_price) / purchase_price
        vol = row[vol_col]
        trade_data.append((ret, vol))
    return trade_data

# Получаем данные по сделкам для всех индексов
trade_data_MOEX = get_trade_returns_with_vol(df_MOEX, "Цена доллары", "Дата", "conditional_volatility", 0.01)
trade_data_SPY = get_trade_returns_with_vol(df_SPY, "Close", "Date", "conditional_volatility", 0.01)
trade_data_HKE = get_trade_returns_with_vol(df_HKE, "Цена доллары", "Дата", "conditional_volatility", 0.01)
trade_data_GOLD = get_trade_returns_with_vol(df_GOLD, "Цена", "Дата", "conditional_volatility", 0.01)

# Расчитаем доходности и волатильности
trade_returns_MOEX = np.array([ret for ret, vol in trade_data_MOEX])
trade_returns_SPY = np.array([ret for ret, vol in trade_data_SPY])
trade_returns_HKE = np.array([ret for ret, vol in trade_data_HKE])
trade_returns_GOLD = np.array([ret for ret, vol in trade_data_GOLD])

vol_MOEX = np.array([vol for ret, vol in trade_data_MOEX])
vol_SPY = np.array([vol for ret, vol in trade_data_SPY])
vol_HKE = np.array([vol for ret, vol in trade_data_HKE])
vol_GOLD = np.array([vol for ret, vol in trade_data_GOLD])

# Рассчитываем историческую и текущую волатильность
historical_vol_MOEX = vol_MOEX.mean() if len(vol_MOEX) > 0 else 1.0
historical_vol_SPY = vol_SPY.mean() if len(vol_SPY) > 0 else 1.0
historical_vol_HKE = vol_HKE.mean() if len(vol_HKE) > 0 else 1.0
historical_vol_GOLD = vol_GOLD.mean() if len(vol_GOLD) > 0 else 1.0

current_vol_MOEX = df_MOEX['conditional_volatility'].iloc[-1] if len(df_MOEX) > 0 else historical_vol_MOEX
current_vol_SPY = df_SPY['conditional_volatility'].iloc[-1] if len(df_SPY) > 0 else historical_vol_SPY
current_vol_HKE = df_HKE['conditional_volatility'].iloc[-1] if len(df_HKE) > 0 else historical_vol_HKE
current_vol_GOLD = df_GOLD['conditional_volatility'].iloc[-1] if len(df_GOLD) > 0 else historical_vol_GOLD

# Модифицированный бутстреппинг с учетом волатильности
iterations = 1000
portfolio_value = 1.0  # Для расчета expected_profit

bootstrap_results = []

# Перебор всех возможных весов для 4 активов
for w_MOEX in range(1, 100):
    for w_SPY in range(1, 100 - w_MOEX):
        w_HKE = 100 - w_MOEX - w_SPY
        w_MOEX_dec = w_MOEX / 100.0
        w_SPY_dec = w_SPY / 100.0
        w_HKE_dec = w_HKE / 100.0
        w_GOLD_dec = 1.0 - (w_MOEX_dec + w_SPY_dec + w_HKE_dec)  # Остаток для GOLD

        portfolio_returns = []

        for _ in range(iterations):
            # Корректируем доходности на текущую волатильность
            adj_ret_MOEX = np.random.choice(trade_returns_MOEX) * (current_vol_MOEX / historical_vol_MOEX) if trade_returns_MOEX.size > 0 else 0
            adj_ret_SPY = np.random.choice(trade_returns_SPY) * (current_vol_SPY / historical_vol_SPY) if trade_returns_SPY.size > 0 else 0
            adj_ret_HKE = np.random.choice(trade_returns_HKE) * (current_vol_HKE / historical_vol_HKE) if trade_returns_HKE.size > 0 else 0
            adj_ret_GOLD = np.random.choice(trade_returns_GOLD) * (current_vol_GOLD / historical_vol_GOLD) if trade_returns_GOLD.size > 0 else 0

            portfolio_ret = w_MOEX_dec * adj_ret_MOEX + w_SPY_dec * adj_ret_SPY + w_HKE_dec * adj_ret_HKE + w_GOLD_dec * adj_ret_GOLD
            portfolio_returns.append(portfolio_ret)

        portfolio_returns = np.array(portfolio_returns)
        VaR_port = np.percentile(portfolio_returns, 5)
        tail_losses = portfolio_returns[portfolio_returns <= VaR_port]
        CVaR_port = -np.mean(tail_losses) * portfolio_value if tail_losses.size > 0 else 0

        mu_port = w_MOEX_dec * trade_returns_MOEX.mean() + w_SPY_dec * trade_returns_SPY.mean() + w_HKE_dec * trade_returns_HKE.mean() + w_GOLD_dec * trade_returns_GOLD.mean() if trade_returns_MOEX.size > 0 and trade_returns_SPY.size > 0 and trade_returns_HKE.size > 0 and trade_returns_GOLD.size > 0 else 0
        expected_profit = mu_port * portfolio_value
        k = expected_profit - CVaR_port

        bootstrap_results.append((k, w_MOEX, w_SPY, w_HKE, w_GOLD))

# Находим оптимальное распределение
max_k, best_w_MOEX, best_w_SPY, best_w_HKE, best_w_GOLD = max(bootstrap_results, key=lambda x: x[0])

print("\nОптимальное распределение с учетом GARCH-волатильности для 4 активов:")
print(f"MOEX: {best_w_MOEX}%, SPY: {best_w_SPY}%, HKE: {best_w_HKE}%, GOLD: {best_w_GOLD}%, Ожидаемый доход-CVaR: {max_k:.2f}")



Оптимальное распределение с учетом GARCH-волатильности для 4 активов:
MOEX: 86%, SPY: 1%, HKE: 13%, GOLD: 2%, Ожидаемый доход-CVaR: 0.32


In [ ]:
# что будем сравнивать
candidate_methods = [
    'historical_VaR',
    'parametric_VaR',
    'monte_carlo_VaR',
    'bootstrap_CVaR',
    'parametric_CVaR',
    'garch_CVaR'
]
def calculate_historical_VaR(portfolio_returns, portfolio_value, confidence_level=0.95):

    alpha_percentile = (1 - confidence_level) * 100
    var = np.percentile(portfolio_returns, alpha_percentile)
    portfolio_var = -var * portfolio_value
    return portfolio_var

def calculate_parametric_VaR(df, price_col, date_col,
                             portfolio_value=1000000,
                             confidence_level=0.95,
                             tolerance=0.01):
    returns = df['Returns'].dropna() if 'Returns' in df.columns else df['Returns_in_dollars'].dropna()
    if len(returns) < 2:
        return None

    mu = returns.mean()
    sigma = returns.std()
    z = norm.ppf(confidence_level)
    critical_return = mu - z * sigma
    VaR_param = -critical_return * portfolio_value
    return VaR_param, mu, sigma
#массивы исторических доходностей и всякого прочего под что будем подгонять и сравнивать

trade_returns_MOEX = np.array([])
trade_returns_SPY  = np.array([])
trade_returns_HKE  = np.array([])
trade_returns_GOLD = np.array([])
mu_MOEX, sigma_MOEX = 0, 0
mu_SPY,  sigma_SPY  = 0, 0
mu_HKE,  sigma_HKE  = 0, 0
mu_GOLD, sigma_GOLD = 0, 0
historical_vol_MOEX, current_vol_MOEX = 1.0, 1.0
historical_vol_SPY,  current_vol_SPY  = 1.0, 1.0
historical_vol_HKE,  current_vol_HKE  = 1.0, 1.0
historical_vol_GOLD, current_vol_GOLD = 1.0, 1.0
def compute_optimal_allocation(window_data,
                               method,
                               portfolio_value=1000000,
                               confidence_level=0.95,
                               tolerance=0.01):

    #ищем весовые коэффициенты активов,максимизирующие (ожидаемый доход - риск).
    global trade_returns_MOEX, trade_returns_SPY, trade_returns_HKE, trade_returns_GOLD
    global mu_MOEX, sigma_MOEX, mu_SPY, sigma_SPY
    global mu_HKE, sigma_HKE, mu_GOLD, sigma_GOLD
    global historical_vol_MOEX, current_vol_MOEX
    global historical_vol_SPY,  current_vol_SPY
    global historical_vol_HKE,  current_vol_HKE
    global historical_vol_GOLD, current_vol_GOLD

    steps = np.linspace(0, 1, 11)
    best_k = -np.inf
    best_weights = np.array([0.25, 0.25, 0.25, 0.25])
    # я тут еще раз прописал все функции , так бтв удобнее
    if method == 'historical_VaR':
        returns_MOEX = window_data['MOEX']['Returns_in_dollars'].dropna().reset_index(drop=True)
        returns_SPY  = window_data['SPY']['Returns'].dropna().reset_index(drop=True)
        returns_HKE  = window_data['HKE']['Returns_in_dollars'].dropna().reset_index(drop=True)
        returns_GOLD = window_data['GOLD']['Returns'].dropna().reset_index(drop=True)

        n = min(len(returns_MOEX), len(returns_SPY), len(returns_HKE), len(returns_GOLD))
        if n == 0:
            return best_k, best_weights

        returns_MOEX = returns_MOEX.iloc[:n]
        returns_SPY  = returns_SPY.iloc[:n]
        returns_HKE  = returns_HKE.iloc[:n]
        returns_GOLD = returns_GOLD.iloc[:n]

        for w_MOEX in steps:
            for w_SPY in steps:
                for w_HKE in steps:
                    w_GOLD = 1 - (w_MOEX + w_SPY + w_HKE)
                    if w_GOLD < 0:
                        continue
                    portfolio_returns = (w_MOEX * returns_MOEX +
                                         w_SPY  * returns_SPY +
                                         w_HKE  * returns_HKE +
                                         w_GOLD * returns_GOLD)
                    expected_profit = portfolio_returns.mean() * portfolio_value
                    portfolio_VaR  = calculate_historical_VaR(portfolio_returns, portfolio_value, confidence_level)
                    k_val = expected_profit - portfolio_VaR
                    if k_val > best_k:
                        best_k = k_val
                        best_weights = np.array([w_MOEX, w_SPY, w_HKE, w_GOLD])
        return best_k, best_weights
    elif method == 'parametric_VaR':
        res_MOEX = calculate_parametric_VaR(window_data['MOEX'], "Цена доллары", "Дата",
                                            portfolio_value, confidence_level, tolerance)
        res_SPY  = calculate_parametric_VaR(window_data['SPY'],  "Close",       "Date",
                                            portfolio_value, confidence_level, tolerance)
        res_HKE  = calculate_parametric_VaR(window_data['HKE'],  "Цена доллары","Дата",
                                            portfolio_value, confidence_level, tolerance)
        res_GOLD = calculate_parametric_VaR(window_data['GOLD'], "Цена",        "Дата",
                                            portfolio_value, confidence_level, tolerance)
        if None in (res_MOEX, res_SPY, res_HKE, res_GOLD):
            return best_k, best_weights

        _, muMOEX, sigmaMOEX = res_MOEX
        _, muSPY,  sigmaSPY  = res_SPY
        _, muHKE,  sigmaHKE  = res_HKE
        _, muGOLD, sigmaGOLD = res_GOLD

        z = norm.ppf(confidence_level)

        for w_MOEX in steps:
            for w_SPY in steps:
                for w_HKE in steps:
                    w_GOLD = 1 - (w_MOEX + w_SPY + w_HKE)
                    if w_GOLD < 0:
                        continue
                    portfolio_mu = (w_MOEX * muMOEX + w_SPY * muSPY +
                                    w_HKE * muHKE  + w_GOLD * muGOLD)
                    portfolio_sigma = np.sqrt(
                        (w_MOEX * sigmaMOEX)**2 +
                        (w_SPY  * sigmaSPY )**2 +
                        (w_HKE  * sigmaHKE )**2 +
                        (w_GOLD * sigmaGOLD)**2
                    )
                    critical_return = portfolio_mu - z * portfolio_sigma
                    portfolio_VaR = -critical_return * portfolio_value
                    expected_profit = portfolio_mu * portfolio_value
                    k_val = expected_profit - portfolio_VaR
                    if k_val > best_k:
                        best_k = k_val
                        best_weights = np.array([w_MOEX, w_SPY, w_HKE, w_GOLD])
        return best_k, best_weights
    elif method == 'monte_carlo_VaR':
        iterations = 50
        alpha_percentile = (1 - confidence_level) * 100

        res_MOEX = calculate_parametric_VaR(window_data['MOEX'], "Цена доллары", "Дата",
                                            portfolio_value, confidence_level, tolerance)
        res_SPY  = calculate_parametric_VaR(window_data['SPY'],  "Close",       "Date",
                                            portfolio_value, confidence_level, tolerance)
        res_HKE  = calculate_parametric_VaR(window_data['HKE'],  "Цена доллары","Дата",
                                            portfolio_value, confidence_level, tolerance)
        res_GOLD = calculate_parametric_VaR(window_data['GOLD'], "Цена",        "Дата",
                                            portfolio_value, confidence_level, tolerance)
        if None in (res_MOEX, res_SPY, res_HKE, res_GOLD):
            return best_k, best_weights

        _, muMOEX, sigmaMOEX = res_MOEX
        _, muSPY,  sigmaSPY  = res_SPY
        _, muHKE,  sigmaHKE  = res_HKE
        _, muGOLD, sigmaGOLD = res_GOLD

        for w_MOEX in steps:
            for w_SPY in steps:
                for w_HKE in steps:
                    w_GOLD = 1 - (w_MOEX + w_SPY + w_HKE)
                    if w_GOLD < 0:
                        continue
                    sim_MOEX = np.random.normal(muMOEX, sigmaMOEX, iterations)
                    sim_SPY  = np.random.normal(muSPY,  sigmaSPY,  iterations)
                    sim_HKE  = np.random.normal(muHKE,  sigmaHKE,  iterations)
                    sim_GOLD = np.random.normal(muGOLD, sigmaGOLD, iterations)
                    sim_portfolio = (w_MOEX * sim_MOEX + w_SPY * sim_SPY +
                                     w_HKE * sim_HKE   + w_GOLD * sim_GOLD)
                    portfolio_VaR = -np.percentile(sim_portfolio, alpha_percentile) * portfolio_value
                    portfolio_mu = (w_MOEX * muMOEX + w_SPY * muSPY +
                                    w_HKE * muHKE   + w_GOLD * muGOLD)
                    expected_profit = portfolio_mu * portfolio_value
                    k_val = expected_profit - portfolio_VaR
                    if k_val > best_k:
                        best_k = k_val
                        best_weights = np.array([w_MOEX, w_SPY, w_HKE, w_GOLD])
        return best_k, best_weights
    elif method == 'bootstrap_CVaR':
        if trade_returns_MOEX.size == 0:
            trade_returns_MOEX = np.array([0])
        if trade_returns_SPY.size == 0:
            trade_returns_SPY = np.array([0])
        if trade_returns_HKE.size == 0:
            trade_returns_HKE = np.array([0])
        if trade_returns_GOLD.size == 0:
            trade_returns_GOLD = np.array([0])

        alpha_percentile = (1 - confidence_level) * 100
        iterations = 50
        bootstrap_results = []

        for w_MOEX in range(1, 97):
            for w_SPY in range(1, 98 - w_MOEX):
                for w_HKE in range(1, 99 - w_MOEX - w_SPY):
                    w_GOLD = 100 - w_MOEX - w_SPY - w_HKE

                    w_MOEX_dec = w_MOEX / 100.0
                    w_SPY_dec  = w_SPY / 100.0
                    w_HKE_dec  = w_HKE / 100.0
                    w_GOLD_dec = w_GOLD / 100.0

                    portfolio_returns = []
                    for _ in range(iterations):
                        s_MOEX = np.random.choice(trade_returns_MOEX)
                        s_SPY  = np.random.choice(trade_returns_SPY)
                        s_HKE  = np.random.choice(trade_returns_HKE)
                        s_GOLD = np.random.choice(trade_returns_GOLD)
                        ret = (w_MOEX_dec * s_MOEX +
                               w_SPY_dec  * s_SPY  +
                               w_HKE_dec  * s_HKE  +
                               w_GOLD_dec * s_GOLD)
                        portfolio_returns.append(ret)

                    portfolio_returns = np.array(portfolio_returns)
                    VaR_port = np.percentile(portfolio_returns, alpha_percentile)
                    tail_losses = portfolio_returns[portfolio_returns <= VaR_port]
                    CVaR_port = -np.mean(tail_losses) * portfolio_value if tail_losses.size > 0 else 0

                    mu_portfolio = (w_MOEX_dec * trade_returns_MOEX.mean() +
                                    w_SPY_dec  * trade_returns_SPY.mean()  +
                                    w_HKE_dec  * trade_returns_HKE.mean()  +
                                    w_GOLD_dec * trade_returns_GOLD.mean())
                    expected_profit = mu_portfolio * portfolio_value
                    k_val = expected_profit - CVaR_port
                    bootstrap_results.append((k_val, w_MOEX_dec, w_SPY_dec, w_HKE_dec, w_GOLD_dec))

        best_k, bw_MOEX, bw_SPY, bw_HKE, bw_GOLD = max(bootstrap_results, key=lambda x: x[0])
        best_weights = np.array([bw_MOEX, bw_SPY, bw_HKE, bw_GOLD])
        return best_k, best_weights
    elif method == 'parametric_CVaR':
        z = norm.ppf(confidence_level)
        phi_z = norm.pdf(z)
        results_param_CVaR = []

        for w_MOEX in range(1, 97):
            for w_SPY in range(1, 98 - w_MOEX):
                for w_HKE in range(1, 99 - w_MOEX - w_SPY):
                    w_GOLD = 100 - w_MOEX - w_SPY - w_HKE

                    w_MOEX_dec = w_MOEX / 100.0
                    w_SPY_dec  = w_SPY / 100.0
                    w_HKE_dec  = w_HKE / 100.0
                    w_GOLD_dec = w_GOLD / 100.0

                    mu_portfolio = (w_MOEX_dec * mu_MOEX +
                                    w_SPY_dec  * mu_SPY  +
                                    w_HKE_dec  * mu_HKE  +
                                    w_GOLD_dec * mu_GOLD)
                    sigma_portfolio = np.sqrt(
                        (w_MOEX_dec * sigma_MOEX)**2 +
                        (w_SPY_dec  * sigma_SPY )**2 +
                        (w_HKE_dec  * sigma_HKE )**2 +
                        (w_GOLD_dec * sigma_GOLD)**2
                    )
                    cvar_portfolio = -(
                        mu_portfolio - sigma_portfolio * (phi_z / (1 - confidence_level))
                    ) * portfolio_value

                    expected_profit = mu_portfolio * portfolio_value
                    k_val = expected_profit - cvar_portfolio
                    results_param_CVaR.append((k_val, w_MOEX_dec, w_SPY_dec, w_HKE_dec, w_GOLD_dec))

        best_k, bw_MOEX, bw_SPY, bw_HKE, bw_GOLD = max(results_param_CVaR, key=lambda x: x[0])
        best_weights = np.array([bw_MOEX, bw_SPY, bw_HKE, bw_GOLD])
        return best_k, best_weights
    elif method == 'garch_CVaR':
        alpha_percentile = (1 - confidence_level) * 100
        iterations = 100
        bootstrap_results = []

        for w_MOEX in range(1, 100):
            for w_SPY in range(1, 100 - w_MOEX):
                w_HKE = 100 - w_MOEX - w_SPY
                w_MOEX_dec = w_MOEX / 100.0
                w_SPY_dec  = w_SPY / 100.0
                w_HKE_dec  = w_HKE / 100.0
                w_GOLD_dec = 1.0 - (w_MOEX_dec + w_SPY_dec + w_HKE_dec)

                portfolio_returns = []
                for _ in range(iterations):
                    adj_ret_MOEX = 0
                    if trade_returns_MOEX.size > 0:
                        adj_ret_MOEX = (np.random.choice(trade_returns_MOEX)
                                        * (current_vol_MOEX / historical_vol_MOEX))
                    adj_ret_SPY = 0
                    if trade_returns_SPY.size > 0:
                        adj_ret_SPY = (np.random.choice(trade_returns_SPY)
                                       * (current_vol_SPY / historical_vol_SPY))
                    adj_ret_HKE = 0
                    if trade_returns_HKE.size > 0:
                        adj_ret_HKE = (np.random.choice(trade_returns_HKE)
                                       * (current_vol_HKE / historical_vol_HKE))
                    adj_ret_GOLD = 0
                    if trade_returns_GOLD.size > 0:
                        adj_ret_GOLD = (np.random.choice(trade_returns_GOLD)
                                        * (current_vol_GOLD / historical_vol_GOLD))

                    ret = (w_MOEX_dec * adj_ret_MOEX +
                           w_SPY_dec  * adj_ret_SPY  +
                           w_HKE_dec  * adj_ret_HKE  +
                           w_GOLD_dec * adj_ret_GOLD)
                    portfolio_returns.append(ret)

                portfolio_returns = np.array(portfolio_returns)
                VaR_port = np.percentile(portfolio_returns, alpha_percentile)
                tail_losses = portfolio_returns[portfolio_returns <= VaR_port]
                CVaR_port = -np.mean(tail_losses) if tail_losses.size > 0 else 0

                mu_port = 0
                if trade_returns_MOEX.size > 0:
                    mu_port += w_MOEX_dec * trade_returns_MOEX.mean()
                if trade_returns_SPY.size > 0:
                    mu_port += w_SPY_dec  * trade_returns_SPY.mean()
                if trade_returns_HKE.size > 0:
                    mu_port += w_HKE_dec  * trade_returns_HKE.mean()
                if trade_returns_GOLD.size > 0:
                    mu_port += w_GOLD_dec * trade_returns_GOLD.mean()

                expected_profit = mu_port
                k_val = expected_profit - CVaR_port

                bootstrap_results.append((k_val, w_MOEX_dec, w_SPY_dec, w_HKE_dec, w_GOLD_dec))

        best_k, bw_MOEX, bw_SPY, bw_HKE, bw_GOLD = max(bootstrap_results, key=lambda x: x[0])
        best_weights = np.array([bw_MOEX, bw_SPY, bw_HKE, bw_GOLD])
        return best_k, best_weights

    else:
        return -np.inf, np.array([0.25, 0.25, 0.25, 0.25])


# перебираем все методы
def compute_best_strategy(window_data):
    best_k = -np.inf
    best_method = None
    best_weights = None
    for method in candidate_methods:
        k, weights = compute_optimal_allocation(window_data, method)
        if k > best_k:
            best_k = k
            best_method = method
            best_weights = weights
    return best_method, best_weights

#обучающий набор
def extract_features(window_data):
    feature_list = []
    for asset, df in window_data.items():
        if asset in ['MOEX', 'HKE']:
            returns = df['Returns_in_dollars'].dropna()
        else:
            returns = df['Returns'].dropna()
        mean_ret = returns.mean()
        std_ret  = returns.std()
        feature_list.extend([mean_ret, std_ret])
    return np.array(feature_list)

cached_results = {}

def get_cache_key(window_data):
    """Создаем хеш от набора данных, чтобы использовать как ключ кэша"""
    key_string = "".join([df.to_string(index=False) for df in window_data.values()])
    return hashlib.md5(key_string.encode()).hexdigest()
def process_window(window_data, candidate_methods, portfolio_value=1000000):
    cache_key = get_cache_key(window_data)
    if cache_key in cached_results:
        return cached_results[cache_key]

    best_k = -np.inf
    best_method = None
    best_weights = None

    for method in candidate_methods:
        k, weights = compute_optimal_allocation(
            window_data, method,
            portfolio_value=portfolio_value,
            confidence_level=0.95,
            tolerance=0.01
        )
        if k > best_k:
            best_k = k
            best_method = method
            best_weights = weights

    features = extract_features(window_data)
    cached_results[cache_key] = (features, best_method, best_weights)
    return features, best_method, best_weights


def generate_dataset_parallel(df_MOEX, df_SPY, df_HKE, df_GOLD,
                              window_size=365, step=45,
                              candidate_methods=None,
                              show_progress=True):
    X, y_method, y_weights = [], [], []

    start_date = pd.to_datetime("1997-01-01")
    end_date = pd.to_datetime("2008-12-31")
    current_date = start_date + pd.Timedelta(days=window_size)

    windows = []
    while current_date <= end_date:
        window_start = current_date - pd.Timedelta(days=window_size)
        window_data = {
            'MOEX': df_MOEX[(df_MOEX['Дата'] >= window_start) & (df_MOEX['Дата'] <= current_date)],
            'SPY':  df_SPY[(df_SPY['Date']   >= window_start) & (df_SPY['Date']   <= current_date)],
            'HKE':  df_HKE[(df_HKE['Дата']   >= window_start) & (df_HKE['Дата']   <= current_date)],
            'GOLD': df_GOLD[(df_GOLD['Дата'] >= window_start) & (df_GOLD['Дата'] <= current_date)]
        }
        if all(len(df_) >= window_size * 0.6 for df_ in window_data.values()):
            windows.append(window_data)
        current_date += pd.Timedelta(days=step)

    iterator = tqdm(windows, desc="Обработка окон") if show_progress else windows

    results = Parallel(n_jobs=-1)(
        delayed(process_window)(win, candidate_methods) for win in iterator
    )

    for features, best_method, best_weights in results:
        X.append(features)
        y_method.append(candidate_methods.index(best_method))
        y_weights.append(best_weights)

    return np.array(X), np.array(y_method), np.array(y_weights)
#обучение+ предсказение
def main_training_and_prediction(df_MOEX, df_SPY, df_HKE, df_GOLD):
    X, y_method, y_weights = generate_dataset_parallel(df_MOEX, df_SPY, df_HKE, df_GOLD,
                                                   window_size=365, step=45,
                                                   candidate_methods=candidate_methods,
                                                   show_progress=True)
    print("Размер X:", X.shape)
    print("Развмер  y_method:", y_method.shape)
    print("Размер  y_weights:", y_weights.shape)

    X_train, X_temp, y_method_train, y_method_temp, y_weights_train, y_weights_temp = train_test_split(
        X, y_method, y_weights, test_size=0.3, random_state=42
    )
    X_val, X_test, y_method_val, y_method_test, y_weights_val, y_weights_test = train_test_split(
        X_temp, y_method_temp, y_weights_temp, test_size=0.5, random_state=42
    )

    print("Train размер :", X_train.shape)
    print("Validation size:", X_val.shape)
    print("Test размер :", X_test.shape)

    input_dim = X_train.shape[1]
    num_methods = len(candidate_methods)
    num_assets = 4

    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dense(32, activation='relu')(x)

    method_output = layers.Dense(num_methods, activation='softmax', name='method_output')(x)
    weights_output = layers.Dense(num_assets, activation='softmax', name='weights_output')(x)

    model = models.Model(inputs=inputs, outputs=[method_output, weights_output])
    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.001),
        loss={'method_output': 'sparse_categorical_crossentropy',
              'weights_output': 'mse'},
        loss_weights={'method_output': 1.0, 'weights_output': 1.0},
        metrics={'method_output': 'accuracy', 'weights_output': 'mse'}
    )
    model.summary()

    history = model.fit(
        X_train,
        {'method_output': y_method_train, 'weights_output': y_weights_train},
        epochs=20,
        batch_size=16,
        validation_data=(X_val, {'method_output': y_method_val, 'weights_output': y_weights_val})
    )

    results = model.evaluate(
        X_test,
        {'method_output': y_method_test, 'weights_output': y_weights_test}
    )
    print("Test results:", results)
    end_date = min(df_MOEX['Дата'].max(), df_SPY['Date'].max(),
                   df_HKE['Дата'].max(), df_GOLD['Дата'].max())
    window_start = end_date - pd.Timedelta(days=365)

    new_window_data = {
        'MOEX': df_MOEX[(df_MOEX['Дата'] >= window_start) & (df_MOEX['Дата'] <= end_date)],
        'SPY':  df_SPY[(df_SPY['Date']   >= window_start) & (df_SPY['Date']   <= end_date)],
        'HKE':  df_HKE[(df_HKE['Дата']   >= window_start) & (df_HKE['Дата']   <= end_date)],
        'GOLD': df_GOLD[(df_GOLD['Дата'] >= window_start) & (df_GOLD['Дата'] <= end_date)]
    }

    sample_features = extract_features(new_window_data).reshape(1, -1)
    pred_method_logits, pred_weights = model.predict(sample_features)
    pred_method_idx = np.argmax(pred_method_logits, axis=1)[0]
    pred_method_name = candidate_methods[pred_method_idx]

    print("Результат на последнем году данных")
    print("Предсказанный метод оценки:", pred_method_name)
    print("Предсказанное распределение активов:", pred_weights[0])
    print("Сумма весов:", pred_weights[0].sum())

    # Сравним с прямым переьбором
    best_m, best_w = compute_best_strategy(new_window_data)
    print("По прямому перебору: метод =", best_m, "веса =", best_w)

    return model





In [ ]:
from tqdm import tqdm

# обучение
model = main_training_and_prediction(df_MOEX, df_SPY, df_HKE, df_GOLD)

# окно в 365
end_date_2012 = pd.to_datetime("2012-12-31")
window_start_2013 = end_date_2012 - pd.Timedelta(days=365)

new_window_data_2013 = {
    'MOEX': df_MOEX[(df_MOEX['Дата'] >= window_start_2013) & (df_MOEX['Дата'] <= end_date_2012)],
    'SPY':  df_SPY[(df_SPY['Date']   >= window_start_2013) & (df_SPY['Date']   <= end_date_2012)],
    'HKE':  df_HKE[(df_HKE['Дата']   >= window_start_2013) & (df_HKE['Дата']   <= end_date_2012)],
    'GOLD': df_GOLD[(df_GOLD['Дата'] >= window_start_2013) & (df_GOLD['Дата'] <= end_date_2012)]
}

# проверим все ли норм
if all(len(df_) >= 365 * 0.6 for df_ in new_window_data_2013.values()):
    print("\nИзвлечение признаков и предсказание модели...")
    sample_features_2013 = extract_features(new_window_data_2013).reshape(1, -1)
    pred_method_logits_2013, pred_weights_2013 = model.predict(sample_features_2013)
    pred_method_idx_2013 = np.argmax(pred_method_logits_2013, axis=1)[0]
    pred_method_name_2013 = candidate_methods[pred_method_idx_2013]

    print("Результат на последнем году данных (для начала 2013 года)")
    print("Предсказанный метод оценки:", pred_method_name_2013)
    print("Предсказанное распределение активов:", pred_weights_2013[0])
    print("Сумма весов:", pred_weights_2013[0].sum())

    print("\nВыполняется прямой перебор...")
    best_m_2013, best_w_2013 = compute_best_strategy(new_window_data_2013)
    print("По прямому перебору для начала 2013 года: метод =", best_m_2013, "веса =", best_w_2013)
else:
    print("Недостаточно данных для прогноза на начало 2013 года.")


Обработка окон: 100%|██████████| 76/76 [9:38:40<00:00, 456.85s/it]


Shape of X: (76, 8)
Shape of y_method: (76,)
Shape of y_weights: (76, 4)
Train size: (53, 8)
Validation size: (11, 8)
Test size: (12, 8)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 8)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            576 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ method_output (Dense)     │ (None, 6)              │            198 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ weights_output (Dense)    │ (None, 4)              │            132 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,986 (11.66 KB)

 Trainable params: 2,986 (11.66 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 135ms/step - loss: 1.8928 - method_output_accuracy: 0.0000e+00 - method_output_loss: 1.8032 - weights_output_loss: 0.0792 - weights_output_mse: 0.0785 - val_loss: 1.6627 - val_method_output_accuracy: 0.0000e+00 - val_method_output_loss: 1.5917 - val_weights_output_loss: 0.0710 - val_weights_output_mse: 0.0710
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 1.5883 - method_output_accuracy: 0.3158 - method_output_loss: 1.5095 - weights_output_loss: 0.0696 - weights_output_mse: 0.0696 - val_loss: 1.4161 - val_method_output_accuracy: 1.0000 - val_method_output_loss: 1.3504 - val_weights_output_loss: 0.0657 - val_weights_output_mse: 0.0657
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 1.3482 - method_output_accuracy: 1.0000 - method_output_loss: 1.2760 - weights_output_loss: 0.0678 - weights_output_mse: 0.0672 - val_loss: 1.1986 - val_method_output_accuracy: 1.0000 - val_method_output_loss: 1.1381 - val_weights_output_loss: 0.0605 -

In [ ]:
model.save("my_trained_model.keras")
# отдельный сэйв параметров
import joblib
joblib.dump({
    'extract_features': extract_features,
    'compute_best_strategy': compute_best_strategy,
    'candidate_methods': candidate_methods
}, "model_helpers.pkl")

['model_helpers.pkl']